In [ ]:
import imageio.v3 as iio
import skimage as ski
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

image = iio.imread(uri='data/colonies-01.tif')
gray_image = ski.color.rgb2gray(image)
blurred_image = ski.filters.gaussian(gray_image, sigma=1)

histogram, bin_edges = np.histogram(blurred_image, bins=256, range=(0, 1))
# 0.2 is a good threshold for this image
threshold = 0.2

binary_mask = blurred_image < threshold
labeled_image, count = ski.measure.label(
    binary_mask, return_num=True)
print(f'Number of colonies: {count}')

colored_label_image = ski.color.label2rgb(labeled_image, bg_label=0)

summary_image = ski.color.gray2rgb(gray_image)
summary_image[binary_mask] = colored_label_image[binary_mask]

fig, ax = plt.subplots()
ax.imshow(summary_image, cmap='viridis')

In [ ]:
def count_colonies(uri, sigma=1, threshold=0.2):
    image = iio.imread(uri=uri)
    gray_image = ski.color.rgb2gray(image)
    blurred_image = ski.filters.gaussian(gray_image, sigma=sigma)
    binary_mask = blurred_image < threshold
    labeled_image, count = ski.measure.label(
        binary_mask, return_num=True, connectivity=1)
    
    colored_label_image = ski.color.label2rgb(labeled_image, bg_label=0)
    summary_image = ski.color.gray2rgb(gray_image)
    summary_image[binary_mask] = colored_label_image[binary_mask]

    fig, ax = plt.subplots()
    ax.imshow(summary_image)
    return count


In [ ]:
import glob

files = glob.glob('data/colonies-*.tif')
for image in files:
    count = count_colonies(uri=image)
    print(f'{image}: {count} colonies')

In [ ]:
def count_colonies(uri, sigma=1, minimum_area=100):
    image = iio.imread(uri=uri)
    
    gray_image = ski.color.rgb2gray(image)
    
    blurred_image = ski.filters.gaussian(gray_image, sigma=sigma)
    binary_mask = blurred_image < 0.9
    threshold = ski.filters.threshold_otsu(blurred_image[binary_mask])
    binary_mask = np.logical_and(binary_mask, blurred_image < threshold)
    mask = ski.morphology.remove_small_objects(binary_mask, min_size=minimum_area)
    labeled_image, count = ski.measure.label(
        mask, return_num=True, connectivity=1)
    
    colored_label_image = ski.color.label2rgb(labeled_image, bg_label=0)
    
    summary_image = image.copy() 
    # colored_label_image는 float (0,1) 범위로 색상을 표시하고, 
    # image는 0-255 범위로 표시되어 있어서 255를 곱해준다.
    summary_image[mask] = colored_label_image[mask] * 255

    fig, ax = plt.subplots()
    ax.imshow(summary_image)
    return count

In [ ]:
files = glob.glob('data/colonies-*.tif')
for image in files:
    count = count_colonies(uri=image, minimum_area=10)
    print(f'{image}: {count} colonies')